<a href="https://colab.research.google.com/github/armandoordonez/GenAI/blob/main/Lab_2_fine_tune_generative_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune un modelo de Gen AI Model para resúmenes

En este cuaderno, se hará en fine-tune de un LLM existente de Hugging Face para mejorar el resumen de los diálogos. Utilizará el modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que proporciona un modelo tuneado con instrucciones de alta calidad y puede resumir texto. Para mejorar las inferencias, se usará full fine-tuning y evaluará los resultados con métricas de ROUGE. Luego, se usará Parameter Efficient Fine-Tuning (PEFT), evaluará el modelo resultante y verá que los beneficios de PEFT superan las desventajas de rendimiento.

# Tabla de contenido

- [ 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM](#1)
  - [ 1.1 - Set up Kernel and Required Dependencies](#1.1)
  - [ 1.2 - Load Dataset and LLM](#1.2)
  - [ 1.3 - Test the Model with Zero Shot Inferencing](#1.3)
- [ 2 - Perform Full Fine-Tuning](#2)
  - [ 2.1 - Preprocess the Dialog-Summary Dataset](#2.1)
  - [ 2.2 - Fine-Tune the Model with the Preprocessed Dataset](#2.2)
  - [ 2.3 - Evaluate the Model Qualitatively (Human Evaluation)](#2.3)
  - [ 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#2.4)
- [ 3 - Perform Parameter Efficient Fine-Tuning (PEFT)](#3)
  - [ 3.1 - Setup the PEFT/LoRA model for Fine-Tuning](#3.1)
  - [ 3.2 - Train PEFT Adapter](#3.2)
  - [ 3.3 - Evaluate the Model Qualitatively (Human Evaluation)](#3.3)
  - [ 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#3.4)

<a name='1'></a>
## 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

In [28]:
!python --version

Python 3.12.11


In [29]:
!pip install torch==2.2.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install transformers==4.42.4
!pip install datasets==2.19.1
!pip install evaluate==0.4.5
!pip install peft==0.10.0
!pip install accelerate==0.30.0
!pip install safetensors==0.4.2
!pip install wandb

Looking in indexes: https://download.pytorch.org/whl/cpu


In [30]:
# ✅ Imports principales
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    GenerationConfig,
    TrainingArguments,
    Trainer
)
import evaluate
import pandas as pd
import numpy as np
import time

print("🔍 Verificación de versiones:")
print(f"torch: {torch.__version__}")
import transformers, datasets, peft
print(f"transformers: {transformers.__version__}")
print(f"datasets: {datasets.__version__}")
print(f"evaluate: {evaluate.__version__}")
print(f"peft: {peft.__version__}")


🔍 Verificación de versiones:
torch: 2.2.2+cpu
transformers: 4.42.4
datasets: 2.19.1
evaluate: 0.4.5
peft: 0.10.0


In [31]:
!pip show torch torchdata transformers datasets evaluate rouge_score loralib peft

Name: torch
Version: 2.2.2+cpu
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision
---
Name: torchdata
Version: 0.11.0
Summary: Composable data loading modules for PyTorch
Home-page: https://github.com/pytorch/data
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD
Location: /usr/local/lib/python3.12/dist-packages
Requires: requests, torch, urllib3
Required-by: torchtune
---
Name: transformers
Version: 4.42.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contrib

In [32]:
# verificar que transformers funciona

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Probar con el modelo más pequeño

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Test básico
input_text = "translate English to Spanish: Hello world"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=20)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ T5 funciona: {result}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


✅ T5 funciona: Hallo Welt


<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies

In [33]:
# Importe los componentes necesarios.
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.2'></a>
### 1.2 - Cargar Dataset y LLM

[DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) es un dataset de Hugging Face. contiene 10,000+ dialogos con los correspondientes resúmenes y temas etiquetados manualmente.

In [34]:
dataset = load_dataset("abisee/cnn_dailymail", "3.0.0")

In [35]:
print(dataset["train"].features)

{'article': Value(dtype='string', id=None), 'highlights': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}


Load the pre-trained [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer directly from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-base) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [36]:
# Cargamos el modelo y el tokenizador

model_name='google/flan-t5-small'
# model_name='t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Es posible extraer la cantidad de parámetros del modelo y descubrir cuántos de ellos se pueden entrenar.

In [37]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Parametros entrenables del modelo: {trainable_model_params:,.0f}\n Total de parametros del modelo: {all_model_params:,.0f}\n Porcentaje de parametros entrenables {100 * trainable_model_params / all_model_params:.0f}%"
#print(f'El area es: {area:,.2f}')
print(print_number_of_trainable_model_parameters(original_model))

Parametros entrenables del modelo: 76,961,152
 Total de parametros del modelo: 76,961,152
 Porcentaje de parametros entrenables 100%


<a name='1.3'></a>
### 1.3 - Prueba del modelo con Zero Shot Inferencing

Pruebe el modelo con la inferencia de tiro cero. Puede ver que el modelo tiene dificultades para resumir el diálogo en comparación con el resumen de referencia, pero extrae información importante del texto que indica que el modelo se puede ajustar a la tarea en cuestión.

In [38]:
index = 200

dialogue = dataset['test'][index]['article']
summary = dataset['test'][index]['highlights']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f' PROMPT DE ENTRADA:\n{prompt}')
print(dash_line)
print(f'RESUMEN HUMANO (BASELINE) :\n{summary}\n')
print(dash_line)
print(f'RESUMEN GENERADO POR EL MODELO CON ZERO SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
 PROMPT DE ENTRADA:

Summarize the following conversation.

(CNN)Former New England Patriots star Aaron Hernandez will need to keep his lawyers even after being convicted of murder and other charges in the death of Odin Lloyd. The 25-year-old potentially faces three more trials -- one criminal and two civil actions. Next up is another murder trial in which he is accused of killing two men and wounding another person near a Boston nightclub in July 2012. Prosecutors have said Hernandez fatally shot Daniel de Abreu and Safiro Furtado when he fired into their 2003 BMW.  Another passenger was wounded and two others were uninjured. Hernandez pleaded not guilty at his arraignment. The trial was originally slated for May 28, but Jake Wark, spokesman for the Suffolk County District Attorney's Office, said Wednesday the trial had been postponed and no new date had been set. "We expect to select a

<a name='2'></a>
## 2 - Realizar Full Fine-Tuning

<a name='2.1'></a>
### 2.1 - Preprocesar el dataset Dialog-Summary

Se necesita convertir los pares dialogo-resumen (prompt-response) en instrucciones explícitas para el LLM. Agregar una instrucción al inicio del dialogo como `Summarize the following conversation` y al inicio del resumen agregar `Summary`como se muestra a continuación:

Training prompt (dialogue):
```
Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary:
```

Training response (summary):
```
Both Chris and Antje participated in the conversation.
```

Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

In [39]:
def tokenize_function(example):
    start_prompt = 'Summarize the following article.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["article"]]
    print("Size of prompt list: ", len(prompt))  # Imprimir el tamaño de la lista
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["highlights"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['id', 'article', 'highlights',])

In [40]:
print (type(tokenized_datasets))
first_example = tokenized_datasets["train"].select([0])
print(first_example)
print(first_example['labels'])

<class 'datasets.dataset_dict.DatasetDict'>
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1
})
[[8929, 16023, 2213, 4173, 6324, 12591, 15, 2347, 3996, 1755, 329, 13462, 38, 3, 88, 5050, 507, 2089, 3, 5, 5209, 7556, 845, 3, 88, 65, 150, 1390, 12, 9030, 17, 449, 112, 1723, 550, 3, 5, 6324, 12591, 15, 31, 7, 8783, 45, 166, 874, 16023, 4852, 43, 118, 1213, 16, 2019, 3069, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Para ahorrar algo de tiempo en el laboratorio, submuestreará el conjunto de datos:

In [41]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Compruebe las formas de las tres partes del conjunto de datos:

In [42]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (2872, 2)
Validation: (134, 2)
Test: (115, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 2872
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 134
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 115
    })
})


El dataset de salida esta listo para el fine-tunning.

<a name='2.2'></a>
### 2.2 - Aplicar Fine -Tunning para el modelo con el dataset Preprocesado

Ahora utilice la clase integrada "Trainer" de Hugging Face (consulte la documentación [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Pase el conjunto de datos preprocesado con referencia al modelo original. Los demás parámetros de entrenamiento se encuentran experimentalmente y no es necesario profundizar en ellos por el momento.

In [44]:
output_dir = f'./article-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

import os
os.environ["WANDB_DISABLED"] = "true"

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
max_steps is given, it will override any value given in num_train_epochs


Start training process...



In [45]:
#trainer.train()
#Entrenar una versión completamente tuneada  del modelo toma  horas en una GPU. Para ahorrar tiempo, se puede descargar un punto de control del modelo completamente ajustado para utilizarlo en el resto del notebook The size of the downloaded instruct model is approximately 1GB.

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

Entrenar una versión completamente tuneada  del modelo toma  horas en una GPU. Para ahorrar tiempo, se puede descargar un punto de control del modelo completamente ajustado para utilizarlo en el resto del notebook The size of the downloaded instruct model is approximately 1GB.

In [46]:
# Esto se usa cuando se descarga el modelo de un bucket de S3
# !aws s3 cp --recursive s3://dlai-generative-ai/models/flan-dialogue-summary-checkpoint/ ./flan-dialogue-summary-checkpoint/
#!ls -alh ./flan-dialogue-summary-checkpoint/pytorch_model.bin
#instruct_model = AutoModelForSeq2SeqLM.from_pretrained("./flan-dialogue-summary-checkpoint", torch_dtype=torch.bfloat16)

# Este chekpoint del modelo se descarga de Hugging Face
instruct_model_name='./article-summary-training-1758573709/checkpoint-1'
instruct_model = AutoModelForSeq2SeqLM.from_pretrained( instruct_model_name, torch_dtype=torch.bfloat16)


In [47]:

type(instruct_model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

<a name='2.3'></a>
### 2.3 - Evaluar el modelo cualitativamente (evaluación humana)

Como ocurre con muchas aplicaciones GenAI, un enfoque cualitativo en el que uno se hace la pregunta "¿Mi modelo se comporta como se supone que debe hacerlo" suele ser un buen punto de partida. En el siguiente ejemplo , puede ver cómo el modelo ajustado es capaz de crear un resumen razonable del diálogo en comparación con la incapacidad original de comprender lo que se le pide al modelo.

In [48]:
index = 200
dialogue = dataset['test'][index]['article']
human_baseline_summary = dataset['test'][index]['highlights']

prompt = f"""
Summarize the following article.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'RESUMEN HUMANO (BASELINE):\n{human_baseline_summary}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODEL CON INSTRUCCIONES:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
RESUMEN HUMANO (BASELINE):
Aaron Hernandez has been found guilty in Odin Lloyd's death, but his troubles are not over .
He also faces murder charges in Suffolk County, Massachusetts, but trial was postponed .
In addition, Hernandez will face two civil lawsuits; one is in relation to Suffolk County case .
---------------------------------------------------------------------------------------------------
MODELO ORIGINAL:
Aaron Hernandez is a New England Patriots star who was convicted of murder and other charges in the death of Odin Lloyd.
---------------------------------------------------------------------------------------------------
MODEL CON INSTRUCCIONES:
Aaron Hernandez is facing a double-murder trial in the death of a New England Patriots star.


<a name='2.4'></a>
### 2.4 - Evaluar el modelo cuantitativamente (con la métrica ROUGE)

[ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) Ayuda a cuantificar la validez de los resúmenes generados por los modelos. Compara los resúmenes con un resumen de referencia, generalmente creado por un usuario. Si bien no es perfecto, indica el aumento general en la eficacia del resumen que hemos logrado mediante el ajuste.

In [49]:
!pip install rouge_score

In [50]:
rouge = evaluate.load('rouge')

Genere las salidas para la muestra del conjunto de datos de prueba (solo 10 diálogos y resúmenes para ahorrar tiempo) y guarde los resultados.

In [51]:
dialogues = dataset['test'][0:10]['article']
human_baseline_summaries = dataset['test'][0:10]['highlights']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following article.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries
0,Membership gives the ICC jurisdiction over all...,The Palestinian Authority has officially forma...,The Palestinian Authority has officially forma...
1,"Theia, a bully breed mix, was apparently hit b...","Theia, a stray dog, has been buried in a field...","Theia, a stray dog, has been buried in a field..."
2,Mohammad Javad Zarif has spent more time with ...,Mohammad Javad Zarif is the Iranian foreign mi...,Mohammad Javad Zarif is the Iranian foreign mi...
3,17 Americans were exposed to the Ebola virus w...,A Nebraska Medicine spokesman says five Americ...,A Nebraska Medicine spokesman says five Americ...
4,Student is no longer on Duke University campus...,Duke University has admitted to hanging a noos...,Duke University has admitted to hanging a noos...
5,College-bound basketball star asks girl with D...,Trey Moses and Ellie Meredith were invited to ...,Trey Moses and Ellie Meredith were invited to ...
6,Amnesty's annual death penalty report catalogs...,"Amnesty International says it is ""shocking"" th...","Amnesty International says it is ""shocking"" th..."
7,Andrew Getty's death appears to be from natura...,"Andrew Getty, 47, has a net worth of $2.1 bill...","Andrew Getty, 47, has a net worth of $2.1 bill..."
8,"Once a super typhoon, Maysak is now a tropical...",Tropical storm Maysak is expected to make land...,Tropical storm Maysak is expected to make land...
9,"Bob Barker returned to host ""The Price Is Righ...","Bob Barker, who hosted ""Lucky Seven"" for the f...","Bob Barker, who hosted ""Lucky Seven"" for the f..."


Evalúe los modelos que calculan las métricas ROUGE. ¡Observe la mejora en los resultados!

In [52]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': 0.26206689932347876, 'rouge2': 0.08510117850943891, 'rougeL': 0.2222768275252066, 'rougeLsum': 0.2413939727972234}
INSTRUCT MODEL:
{'rouge1': 0.25144555821755754, 'rouge2': 0.07844037347442087, 'rougeL': 0.20918622005762388, 'rougeLsum': 0.2299722229491326}


The results show substantial improvement in all ROUGE metrics:

In [54]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE
rouge1: -1.06%
rouge2: -0.67%
rougeL: -1.31%
rougeLsum: -1.14%


<a name='3'></a>
## 3 - Fine tunning Eficiente de Parámetros (PEFT)

Ahora, realicemos un ajuste fino **PEFT** en lugar del ajuste fino completo, como se hizo anteriormente. PEFT es una forma de ajuste fino de instrucciones mucho más eficiente que el ajuste fino completo, con resultados de evaluación comparables, como verá pronto.

PEFT es un término genérico que incluye **Adaptación de Bajo Rango (LoRA)** y ajuste de prompts (¡que NO ES LO MISMO que ingeniería de prompts!). En la mayoría de los casos, cuando alguien habla de PEFT, se refiere a LoRA. LoRA, a un nivel muy alto, permite al usuario ajustar su modelo utilizando menos recursos computacionales (en algunos casos, una sola GPU). Después del ajuste fino para una tarea, caso de uso o inquilino específico con LoRA, el resultado es que el LLM original permanece sin cambios y surge un nuevo "adaptador LoRA". Este adaptador LoRA es mucho más pequeño que el LLM original: aproximadamente un porcentaje de un solo dígito del tamaño del LLM original (MB vs. GB).

No obstante, en el momento de la inferencia, el adaptador LoRA debe reunirse y combinarse con su LLM original para atender la solicitud de inferencia. La ventaja, sin embargo, es que muchos adaptadores LoRA pueden reutilizar el LLM original, lo que reduce los requisitos generales de memoria al atender múltiples tareas y casos de uso.

<a name='3.1'></a>


### 3.1 - Configuración del modelo PEFT/LoRA para el ajuste fino

Debe configurar el modelo PEFT/LoRA para el ajuste fino con un nuevo adaptador de capa/parámetro. Al usar PEFT/LoRA, se congela el LLM subyacente y solo se entrena el adaptador. Observe la configuración de LoRA a continuación. Observe el hiperparámetro de rango (`r`), que define el rango/dimensión del adaptador que se va a entrenar.

In [55]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [56]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 1,376,256
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 2%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [57]:
output_dir = f'./peft-dialogue-article-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
max_steps is given, it will override any value given in num_train_epochs


Now everything is ready to train the PEFT adapter and save the model.



In [58]:
peft_trainer.train()
peft_model_path="./peft-dialogue-article-checkpoint-local"
peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,39.750000


('./peft-dialogue-article-checkpoint-local/tokenizer_config.json',
 './peft-dialogue-article-checkpoint-local/special_tokens_map.json',
 './peft-dialogue-article-checkpoint-local/spiece.model',
 './peft-dialogue-article-checkpoint-local/added_tokens.json',
 './peft-dialogue-article-checkpoint-local/tokenizer.json')

That training was performed on a subset of data. To load a fully trained PEFT model, read a checkpoint of a PEFT model from S3.

In [ ]:
#!aws s3 cp --recursive s3://dlai-generative-ai/models/peft-dialogue-summary-checkpoint/ ./peft-dialogue-summary-checkpoint-from-s3/

Comprueba que el tamaño de este modelo es mucho menor que el LLM original:

In [61]:
!dir /a .\content\peft-dialogue-article-training-1758583404\checkpoint-1

dir: cannot access '/a': No such file or directory
dir: cannot access '.contentpeft-dialogue-article-training-1758583404checkpoint-1': No such file or directory


Prepare este modelo añadiendo un adaptador al modelo FLAN-T5 original. Está configurando `is_trainable=False` porque el plan es realizar inferencia únicamente con este modelo PEFT. Si estuviera preparando el modelo para entrenamiento posterior, debería configurar `is_trainable=True`.

In [62]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-dialogue-article-training-1758583404/checkpoint-1',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [63]:
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 0
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 0%


<a name='3.3'></a>
### 3.3 - Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in sections [1.3](#1.3) and [2.3](#2.3), with the original model, fully fine-tuned and PEFT model.

In [64]:
index = 200
dialogue = dataset['test'][index]['article']
baseline_human_summary = dataset['test'][index]['highlights']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Aaron Hernandez has been found guilty in Odin Lloyd's death, but his troubles are not over .
He also faces murder charges in Suffolk County, Massachusetts, but trial was postponed .
In addition, Hernandez will face two civil lawsuits; one is in relation to Suffolk County case .
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Hernandez's lawyer says he will be arraigned in the trial of the New England Patriots.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
Aaron Hernandez will be arraigned on Wednesday, but he will not be able to return to court.
---------------------------------------------------------------------------------------------------
PEFT MODEL: Aaron Hernandez will be arraigned on Wednesday, but he will not be able to

<a name='3.4'></a>
### 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time).

In [65]:
dialogues = dataset['test'][0:10]['article']
human_baseline_summaries = dataset['test'][0:10]['highlights']

original_model_summaries = []
instruct_model_summaries = []
peft_model_summaries = []

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    human_baseline_text_output = human_baseline_summaries[idx]

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    instruct_model_summaries.append(instruct_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,instruct_model_summaries,peft_model_summaries
0,Membership gives the ICC jurisdiction over all...,The Palestinians signed the Rome Statute in Ja...,The Palestinian Authority has officially forma...,The Palestinian Authority has officially becom...
1,"Theia, a bully breed mix, was apparently hit b...","Theia is a stray dog, and a dog named Theia is...","Theia, a stray dog, has been buried in a field...","Theia, a stray dog, has been buried in a field..."
2,Mohammad Javad Zarif has spent more time with ...,"The two men, who have been a member of the Ira...",Mohammad Javad Zarif is the Iranian foreign mi...,Mohammad Javad Zarif is the Iranian foreign mi...
3,17 Americans were exposed to the Ebola virus w...,Five Americans who were exposed to Ebola have ...,A Nebraska Medicine spokesman said five Americ...,A Nebraska Medicine spokesman said five Americ...
4,Student is no longer on Duke University campus...,Duke University is investigating a student who...,Duke University has admitted to hanging a noos...,Duke University has admitted to hanging a noos...
5,College-bound basketball star asks girl with D...,Trey and Ellie Meredith are both excited about...,Trey Moses and Ellie Meredith were invited to ...,Trey Moses and Ellie Meredith were invited to ...
6,Amnesty's annual death penalty report catalogs...,Amnesty estimates that at least a quarter of p...,"Amnesty International says it is ""shocking"" th...","Amnesty International says it is ""shocking"" th..."
7,Andrew Getty's death appears to be from natura...,"Andrew Getty, grandson of J. Paul Getty, was f...","Andrew Getty, 47, has a net worth of $2.1 bill...","Andrew Getty, 47, has a net worth of $2.1 bill..."
8,"Once a super typhoon, Maysak is now a tropical...",Tropical storm Maysak has been a major threat ...,Tropical storm Maysak is expected to make land...,Tropical storm Maysak is expected to make land...
9,"Bob Barker returned to host ""The Price Is Righ...","Bob Barker, who hosted the show for the first ...","Bob Barker, who hosted ""The Price Is Right"" fo...","Bob Barker, who hosted ""Lucky Seven"" for the f..."



Calcule la puntuación ROUGE para este subconjunto de los datos.

In [66]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.2549182893986622, 'rouge2': 0.07356904328215805, 'rougeL': 0.1972108520475721, 'rougeLsum': 0.2069255714007535}
INSTRUCT MODEL:
{'rouge1': 0.25595289956907474, 'rouge2': 0.09781080267246764, 'rougeL': 0.22994217104051826, 'rougeLsum': 0.24145730805173746}
PEFT MODEL:
{'rouge1': 0.25656234742991624, 'rouge2': 0.08334320341709506, 'rougeL': 0.22416459104409153, 'rougeLsum': 0.23662839385575674}


Notice, that PEFT model results are not too bad, while the training process was much easier!

The results show less of an improvement over full fine-tuning, but the benefits of PEFT typically outweigh the slightly-lower performance metrics.

Calculate the improvement of PEFT over the original model:

In [67]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE
rouge1: 0.16%
rouge2: 0.98%
rougeL: 2.70%
rougeLsum: 2.97%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [68]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: 0.06%
rouge2: -1.45%
rougeL: -0.58%
rougeLsum: -0.48%


Here you see a small percentage decrease in the ROUGE metrics vs. full fine-tuned. However, the training requires much less computing and memory resources (often just a single GPU).